In [14]:
import pandas as pd
import numpy as np
from datetime import timedelta
import iris_core as iris

### 1. Specifiy IRIS version

In [28]:
#Valid choices are : IRIS_1, IRIS_2, or IRIS_3
IRIS_VERSION = 'IRIS_1'

### 2. Specify IRIS raw file

In [1]:
filepath = '../IRIS_FILE_RETRIEVAL/20220322.TXT'

In [7]:
# Create file date object and read file as pandas dataframe
date_string = filename.split('/')[-1].split('.')[0]

raw_file = pd.read_csv(filename, sep=',', names=['timestamp', 'voltage'])
raw_file['timestamp'] = pd.to_datetime(date_string + ' ' + raw_file['timestamp'])

### 3. Parse raw IRIS files into lists based on time interval between measurements

In [8]:
def compute_time_difference(row):
    #Handle first row exception
    if row.name == 0:
        return timedelta(seconds=0)
    else :
        time_difference = row['timestamp'] - raw_file['timestamp'][int(row.name) - 1]
        return time_difference

In [10]:
raw_file['delta_t'] = raw_file.apply(lambda x : compute_time_difference(x), axis=1)

timestamp = raw_file['timestamp'].to_list()
voltage = raw_file['voltage'].to_list()
delta_t = raw_file['delta_t'].to_list()

In [11]:
c_list_volt = []
c_list_time = []
groups_time = []
groups_volt = []

for idx, interval in enumerate(delta_t):
    if interval < pd.to_timedelta('10s'):
        c_list_volt.append(voltage[idx])
        c_list_time.append(timestamp[idx])
    else :
        groups_volt.append(c_list_volt)
        groups_time.append(c_list_time)
        c_list_volt = [voltage[idx]]
        c_list_time = [timestamp[idx]]

### 4. Read measure labels (calib + height) in excel sheet

In [12]:
in_shape_df = pd.read_excel("C:/Users/paulb/Documents/Doctorat/ROGERS_PASS_2022/2022-03-22/Fidelity_20220322_playground.xlsx",
             sheet_name="IRIS_labels")

In [15]:
# Average voltage measurements and generate calibration and measures dataframe
average_volt = [np.mean(measures) for measures in groups_volt]
in_shape_df['voltage'] = average_volt
calib_df  = in_shape_df[in_shape_df['labels'].str.contains('Spectralon', na=False)].copy()
measures_df = in_shape_df[~in_shape_df['labels'].str.contains('Spectralon', na=False)].copy()

### 5. Process IRIS data

#### Compute reflectance voltage averages for spectralon and measures

In [22]:
calib_df.rename({'labels': 'spectralon'}, inplace=True, axis=1)
calib_df['spectralon'] = calib_df['spectralon'].apply(lambda x : int(x.split(' ')[-1][:-1]))
calib_df = calib_df.groupby(['spectralon']).mean()

#### Create averaged voltages per heights

In [24]:
measures_df.rename({'labels': 'height'}, inplace=True, axis=1)

#### Compute IRIS parameters

In [25]:
# Voltage to reflectance polynomial fit
polynom = iris.calibration_polynom_fit(calib_df.index.values, calib_df['voltage'].values)

In [26]:
# Compute reflectance
measures_df['reflectance'] = measures_df['voltage'].apply(lambda x : iris.voltage_to_reflectance(x, polynom))

In [29]:
# Compute SSA from reflectance
measures_df['ssa'] = measures_df['reflectance'].apply(lambda x : iris.reflectance_to_ssa(x, IRIS_VERSION))

In [30]:
# Compute optical radius from SSA
measures_df['optical radius'] = measures_df['ssa'].apply(lambda x : iris.ssa_to_optical_radius(x))

#### Processed data preview

In [31]:
measures_df.sort_index(ascending=False, inplace=True)
measures_df

,height,voltage,reflectance,ssa,optical radius
39,25,0.484333,1.34,1.44,2.272645
38,30,0.528000,3.50,2.38,1.375045
37,35,0.509667,2.50,1.97,1.661222
36,40,0.521667,3.14,2.24,1.460986
35,45,0.516000,2.83,2.11,1.550999
34,55,0.537667,4.08,2.62,1.249087
33,60,0.535000,3.92,2.55,1.283376
32,65,0.534667,3.90,2.55,1.283376
31,70,0.524667,3.31,2.31,1.416714
30,75,0.514333,2.74,2.07,1.580970


### 5. Write dataframe to a .csv file

In [15]:
measures_df.to_csv(iris_file.split('.')[0] + '_processed.csv')